<a href="https://colab.research.google.com/github/aicuai/Book-SD-MasterGuide/blob/main/ComfyJapan_Flux2_Wan22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [ComfyJapan] ComfyUI を Google Colabで動かす Wan2.2 + FLUX.2

## このノートブックの短縮URLは https://j.aicu.ai/CJCF2  

coded by Akihiko SHIRAI aka しらいはかせ X@o_ob

- 実験用途は L4 GPU で動きますが、実用上は A100 GPU 推奨です。
- FLUX.2 Dev は非商用ライセンスです また [BFLのHuggingFace](https://huggingface.co/black-forest-labs/FLUX.2-dev)で利用許諾に同意が必要です。
- HuggingFaceのトークン `HF_TOKEN`, CivitaiのAPIキーを `CIVITAI_KEY` という名前でシークレットに保存してください（ソースコードに機密を書かない設計です）
- ぶるぺんさんの mellow_pencil-XL もインストールされます(趣味)

原作はざすこさんの [こちら](https://j.aicu.ai/CJCW22Z)↓

🍀[ComfyUI_起動ノート（Wan2_2_フルカスタム版）🌐Drive自動保存付き_20251113.ipynb](https://j.aicu.ai/CJCW22Z)


- ComfyJapan勉強会への参加は[こちら](https://j.aicu.ai/ComfyJapan)

一番下にも謝辞と関連情報へのリンクがあります!

# 🥚各種モデルのDL ＆ ComfyUIの起動🐣
##＜はじめる前の準備＞※重要※
### ❶生成に必要な各種モデルファイルのDL用URLを格納する各フォルダの記入欄に記載して下さい。
└🌐GoogleColab上の 📂ComfyUI / models  以下に、生成に必要な各種ファイルをDLします。（DL容量が少ない程起動が速くなります。）
### ❷（▶）ボタンをクリックしてComfyUIを起動


In [ ]:
# @markdown # 👈この（▶）ボタンをクリックしてComfyUIを起動（☕起動まで約5～6分）
# @markdown ### ※停止と再起動もこのボタンで行います。
# @markdown ## 🌱実行ログに青字で👀「https～ localhost:8188」が表示されたら🖱️クリックしてください。

!pip install torch==2.8.0 torchvision==0.23.0

# using_L4_GPU = True # @param {type:"boolean"}
using_L4_GPU = False
include_manager = True # @param {type:"boolean"}

# ==========================================
# 🔑 シークレットキーの読み込み
# ==========================================
import os
from google.colab import userdata

print("=" * 70)
print("🔑 Checking Secrets...")

# Hugging Face Token (必須)
try:
    hf_token = userdata.get('HF_TOKEN')
    if hf_token:
        os.environ['HF_TOKEN'] = hf_token
        print("✅ HF_TOKEN found in Secrets")
    else:
        print("⚠️ HF_TOKEN not found. Flux.2 downloads will fail.")
except Exception:
    hf_token = ""
    print("⚠️ HF_TOKEN check failed (Secrets not set?)")

# Civitai API Key (任意)
try:
    civitai_key = userdata.get('CIVITAI_KEY')
    if civitai_key:
        print("✅ CIVITAI_KEY found in Secrets")
    else:
        civitai_key = ""
except Exception:
    civitai_key = ""

print("=" * 70)

# ==========================================
# 📁 Google Drive マウント
# ==========================================
# @markdown ## 📁 Google Driveに接続
# @markdown ComfyUIの出力ファイルをバックアップする場合はチェックを入れてください

# @markdown ---
# @markdown ### Google Driveを使用する
use_google_drive = False  # @param {type:"boolean"}

# @markdown ---

if use_google_drive:
    from google.colab import drive
    drive.mount('/content/drive')

    print("=" * 70)
    print("✅ Google Driveがマウントされました")
    print("📁 マウント先: /content/drive/MyDrive")
    print("=" * 70)
else:
    print("=" * 70)
    print("⏸️  Google Driveは使用しません")
    print("💡 出力ファイルは/content/ComfyUI/outputに保存されます")
    print("   （ランタイム終了時に消えるので注意してください）")
    print("=" * 70)


# ==========================================
# 📤 Google Drive 出力設定
# ==========================================
# @markdown ## 📤 Google Drive 出力設定
# @markdown ### ComfyUIの出力先をGoogle Driveに変更する
enable_gdrive_output = False  # @param {type:"boolean"}

# @markdown ### 出力先フォルダ設定
GDRIVE_OUTPUT = "/content/drive/MyDrive/ComfyUI_output"  # @param {type:"string"}

# @markdown ---

# 出力フォルダの作成
if use_google_drive and enable_gdrive_output:
    from pathlib import Path
    output_path = Path(GDRIVE_OUTPUT)

    if output_path.exists():
        print(f"📁 既存のフォルダを使用: {GDRIVE_OUTPUT}")
    else:
        output_path.mkdir(parents=True, exist_ok=True)
        print(f"📁 新規フォルダを作成: {GDRIVE_OUTPUT}")

    print("=" * 70)
    print("✅ ComfyUIの出力先がGoogle Driveに設定されます")
    print(f"📁 保存先: {GDRIVE_OUTPUT}")
    print("💡 生成したファイルが直接Google Driveに保存されます")
    print("=" * 70)
elif enable_gdrive_output and not use_google_drive:
    print("=" * 70)
    print("⚠️ Google Drive出力が有効ですが、Google Driveがマウントされていません")
    print("💡 use_google_driveにチェックを入れてください")
    print("=" * 70)
else:
    print("=" * 70)
    print("⏸️  Google Drive出力は無効です")
    print("💡 出力ファイルは/content/ComfyUI/outputに保存されます")
    print("=" * 70)


%cd /content
from IPython.display import clear_output
clear_output()
!pip install -q torchsde einops diffusers accelerate xformers==0.0.32.post1 triton==3.4
!pip install av spandrel albumentations onnx opencv-python onnxruntime
!pip install color-matcher
!pip install --upgrade "protobuf>=5.29.1"
!pip install onnxruntime-gpu -y
# Install huggingface CLI specifically for Gated/Xet downloads
!pip install -U "huggingface_hub[cli]"

!git clone https://github.com/comfyanonymous/ComfyUI
!pip install -r /content/ComfyUI/requirements.txt
clear_output()

%cd /content/ComfyUI/custom_nodes
# !git clone https://github.com/pythongosssss/ComfyUI-Custom-Scripts.git
if include_manager:
    !git clone https://github.com/ltdrdata/ComfyUI-Manager

# 既存のカスタムノード
!git clone --branch forQwen https://github.com/Isi-dev/ComfyUI_GGUF.git
!git clone https://github.com/Isi-dev/ComfyUI_DeleteModelPassthrough.git
!git clone https://github.com/Isi-dev/comfyui_controlnet_aux
!git clone https://github.com/kijai/ComfyUI-WanVideoWrapper
!git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite
!git clone https://github.com/kijai/ComfyUI-KJNodes.git
!git clone https://github.com/kijai/ComfyUI-segment-anything-2
!git clone https://github.com/kijai/ComfyUI-Florence2
!git clone https://github.com/john-mnz/ComfyUI-Inspyrenet-Rembg.git
!git clone https://github.com/Isi-dev/ComfyUI_Animation_Nodes_and_Workflows

# 🆕 追加のカスタムノード
!git clone https://github.com/rgthree/rgthree-comfy.git
!git clone https://github.com/cubiq/ComfyUI_essentials.git
!git clone https://github.com/chrisgoringe/cg-use-everywhere.git
!git clone https://github.com/aria1th/ComfyUI-LogicUtils.git
!git clone https://github.com/GACLove/ComfyUI-VFI.git

# requirements.txtのインストール
import os
for root, dirs, files in os.walk("/content/ComfyUI/custom_nodes"):
    if "requirements.txt" in files:
        !pip install -r {os.path.join(root, "requirements.txt")}

# 🆕 protobufの最終調整
print("=" * 70)
print("🔧 Fixing protobuf version conflicts...")
!pip install protobuf==5.29.1
print("✅ Protobuf version fixed")
print("=" * 70)

clear_output()

%cd /content/ComfyUI

import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import subprocess
import sys
import shutil
from pathlib import Path

def install_apt_packages():
    packages = ['aria2']
    try:
        subprocess.run(['apt-get', '-y', 'install', '-qq'] + packages, check=True, capture_output=True)
        print("✓ apt packages installed")
    except subprocess.CalledProcessError as e:
        print(f"✗ Error installing apt packages: {e.stderr.decode().strip() or 'Unknown error'}")

print("Installing apt packages...")
install_apt_packages()

# ========================================
# ダウンロード関数群 (Smart Downloader)
# ========================================

def flatten_directory(target_dir):
    """サブディレクトリ内のファイルをルートに移動してディレクトリを削除"""
    target_path = Path(target_dir)
    for file_path in target_path.glob('**/*'):
        if file_path.is_file():
            # ターゲット直下に同名ファイルがない場合のみ移動
            dest = target_path / file_path.name
            if not dest.exists() and file_path.parent != target_path:
                shutil.move(str(file_path), str(dest))

    # 空になったサブディレクトリを削除
    for dir_path in sorted(target_path.glob('**/*'), reverse=True):
        if dir_path.is_dir() and dir_path != target_path:
            try:
                dir_path.rmdir()
            except OSError:
                pass # 空でない場合は無視

def download_with_aria2c(link, folder):
    filename = link.split("/")[-1].split('?')[0]
    print(f"⬇️ Aria2c: {filename}")
    subprocess.run(['aria2c', '--console-log-level=error', '-c', '-x', '16', '-s', '16', '-k', '1M', '-d', folder, '-o', filename, link], check=True)

def download_civitai_model(link, folder, token=None):
    if not token:
        print(f"⚠️ Skipping Civitai link (No Token): {link}")
        return
    try:
        model_id = link.split("/models/")[1].split("?")[0]
    except:
        print(f"❌ Invalid Civitai link: {link}")
        return

    api_url = f"https://civitai.com/api/download/models/{model_id}?type=Model&format=SafeTensor&token={token}"
    print(f"⬇️ Civitai: ID {model_id}")
    subprocess.run(['wget', '--content-disposition', '--show-progress', '-q', '-P', folder, api_url], check=False)

def parse_hf_url(url):
    # URLからRepoIDとファイルパスを抽出
    try:
        parts = url.replace("https://huggingface.co/", "").split("/")
        repo_id = f"{parts[0]}/{parts[1]}"

        if "blob" in parts: idx = parts.index("blob")
        elif "resolve" in parts: idx = parts.index("resolve")
        elif "tree" in parts: idx = parts.index("tree")
        else: return None, None

        filename = "/".join(parts[idx+2:])
        return repo_id, filename
    except:
        return None, None

def download_huggingface_cli(url, folder):
    """Hugging Face CLIを使用してダウンロードし、ディレクトリ構造をフラットにする"""
    repo_id, filename = parse_hf_url(url)
    if not repo_id or not filename:
        print(f"⚠️ HF parse error, falling back to aria2c: {url}")
        download_with_aria2c(url, folder)
        return

    print(f"⬇️ HF-CLI: {filename} from {repo_id}")

    # CLIでダウンロード (認証は環境変数 HF_TOKEN を使用)
    cmd = [
        "huggingface-cli", "download",
        repo_id, filename,
        "--local-dir", folder,
        "--local-dir-use-symlinks", "False",
        "--quiet"
    ]

    try:
        subprocess.run(cmd, check=True)
        # split_filesなどが含まれる場合、フォルダ構造ができるためフラット化
        flatten_directory(folder)
    except subprocess.CalledProcessError as e:
        print(f"❌ Download failed: {e}")

def process_urls(url_string, folder):
    urls = [line.strip() for line in url_string.split('\n') if line.strip() and not line.startswith('#')]
    if not urls: return

    os.makedirs(folder, exist_ok=True)
    print(f"\n📂 Processing: {folder}")

    for url in urls:
        if "huggingface.co" in url:
            download_huggingface_cli(url, folder)
        elif "civitai.com" in url:
            download_civitai_model(url, folder, civitai_key)
        else:
            download_with_aria2c(url, folder)

# ========================================
# 📥 各種モデルのダウンロード設定
# ========================================

# @markdown # 📥 モデルダウンロード設定
# @markdown 各フォルダごとに、使用したいモデルのダウンロードURLを入力してください。
# @markdown 複数のURLは改行で区切って入力できます。

# @markdown ---
# @markdown ## 📁 diffusion_models フォルダ
# @markdown 各種メインモデル（動画生成など）
diffusion_models_url_1 = "https://huggingface.co/QuantStack/Wan2.2-I2V-A14B-GGUF/resolve/main/HighNoise/Wan2.2-I2V-A14B-HighNoise-Q8_0.gguf" # @param {type:"string"}
diffusion_models_url_2 = "https://huggingface.co/QuantStack/Wan2.2-I2V-A14B-GGUF/resolve/main/LowNoise/Wan2.2-I2V-A14B-LowNoise-Q8_0.gguf" # @param {type:"string"}
diffusion_models_url_3 = "https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_i2v_high_noise_14B_fp8_scaled.safetensors" # @param {type:"string"}
diffusion_models_url_4 = "https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_i2v_low_noise_14B_fp8_scaled.safetensors" # @param {type:"string"}
diffusion_models_url_5 = "https://huggingface.co/Comfy-Org/flux2-dev/blob/main/split_files/diffusion_models/flux2_dev_fp8mixed.safetensors" # @param {type:"string"}
diffusion_models_url_6 = "" # @param {type:"string"}
diffusion_models_url_7 = "" # @param {type:"string"}
diffusion_models_url_8 = "" # @param {type:"string"}

# 入力されたURLを結合
diffusion_models_urls = "\n".join([
    url for url in [
        diffusion_models_url_1,
        diffusion_models_url_2,
        diffusion_models_url_3,
        diffusion_models_url_4,
        diffusion_models_url_5,
        diffusion_models_url_6,
        diffusion_models_url_7,
        diffusion_models_url_8
    ] if url.strip()
])

# @markdown ---
# @markdown ## 📁 text_encoders フォルダ
# @markdown テキストエンコーダー（UMT5等）
text_encoders_url_1 = "https://huggingface.co/Comfy-Org/flux2-dev/blob/main/split_files/text_encoders/mistral_3_small_flux2_fp8.safetensors" # @param {type:"string"}
text_encoders_url_2 = "https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/text_encoders/umt5_xxl_fp8_e4m3fn_scaled.safetensors" # @param {type:"string"}
text_encoders_url_3 = "" # @param {type:"string"}

# 入力されたURLを結合
text_encoders_urls = "\n".join([url for url in [text_encoders_url_1, text_encoders_url_2, text_encoders_url_3] if url.strip()])

# @markdown ---
# @markdown ## 📁 vae フォルダ
# @markdown VAE（変分オートエンコーダー）
vae_url_1 = "https://huggingface.co/Comfy-Org/flux2-dev/blob/main/split_files/vae/flux2-vae.safetensors" # @param {type:"string"}
vae_url_2 = "https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/vae/wan_2.1_vae.safetensors" # @param {type:"string"}
vae_url_3 = "https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors" # @param {type:"string"}

# 入力されたURLを結合
vae_urls = "\n".join([url for url in [vae_url_1, vae_url_2, vae_url_3] if url.strip()])

# @markdown ---
# @markdown ## 📁 clip_vision フォルダ
# @markdown CLIP Visionモデル（画像理解用）
clip_vision_url_1 = "https://huggingface.co/Isi99999/Wan_Extras/resolve/main/clip_vision_h.safetensors" # @param {type:"string"}
clip_vision_url_2 = "" # @param {type:"string"}
clip_vision_url_3 = "" # @param {type:"string"}

# 入力されたURLを結合
clip_vision_urls = "\n".join([url for url in [clip_vision_url_1, clip_vision_url_2, clip_vision_url_3] if url.strip()])

# @markdown ---
# @markdown ## 📁 loras フォルダ
# @markdown LoRAモデル（高速化、ライティング調整等）

loras_url_1 = "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Lightx2v/lightx2v_T2V_14B_cfg_step_distill_v2_lora_rank128_bf16.safetensors" # @param {type:"string"}
loras_url_2 = "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Lightx2v/lightx2v_I2V_14B_480p_cfg_step_distill_rank128_bf16.safetensors" # @param {type:"string"}
loras_url_3 = "https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/wan2.2_i2v_lightx2v_4steps_lora_v1_low_noise.safetensors" # @param {type:"string"}
loras_url_4 = "https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/wan2.2_i2v_lightx2v_4steps_lora_v1_high_noise.safetensors" # @param {type:"string"}
loras_url_5 = "" # @param {type:"string"}
loras_url_6 = "" # @param {type:"string"}
loras_url_7 = "" # @param {type:"string"}
loras_url_8 = "" # @param {type:"string"}

# URLをまとめる
loras_urls = "\n".join([url for url in [loras_url_1, loras_url_2, loras_url_3, loras_url_4, loras_url_5, loras_url_6, loras_url_7, loras_url_8] if url])

# @markdown ---
# @markdown ## 📁 audio_encoders フォルダ
# @markdown Audio Encoderモデル（音声付き動画生成用）
audio_encoders_url_1 = "" # @param {type:"string"}
audio_encoders_url_2 = "" # @param {type:"string"}
audio_encoders_url_3 = "" # @param {type:"string"}

# URLをまとめる
audio_encoders_urls = "\n".join([url for url in [audio_encoders_url_1, audio_encoders_url_2, audio_encoders_url_3] if url])

# @markdown ---
# @markdown ## 📁 sam2 フォルダ
# @markdown SAM2セグメンテーションモデル
sam2_urls = "https://huggingface.co/Kijai/sam2-safetensors/resolve/main/sam2.1_hiera_small.safetensors" # @param {type:"string"}

# @markdown ---
# @markdown ## 📁 unet フォルダ
# @markdown UNetモデル（Flux等）
unet_urls = "" # @param {type:"string"}

# @markdown ---
# @markdown ## 📁 checkpoints フォルダ
# @markdown チェックポイントモデル
checkpoints_url_1 = "https://huggingface.co/bluepen5805/mellow_pencil-XL/resolve/main/mellow_pencil-XL-v1.0.0-base.safetensors" # @param {type:"string"}
checkpoints_url_2 = "" # @param {type:"string"}
checkpoints_url_3 = "" # @param {type:"string"}
checkpoints_url_4 = "" # @param {type:"string"}
checkpoints_url_5 = "" # @param {type:"string"}

# 入力されたURLを結合
checkpoints_urls = "\n".join([url for url in [checkpoints_url_1, checkpoints_url_2, checkpoints_url_3, checkpoints_url_4, checkpoints_url_5] if url.strip()])

# @markdown ---
# @markdown ## 📁 controlnet フォルダ
# @markdown ControlNetモデル
controlnet_urls = "" # @param {type:"string"}

# @markdown ---
# @markdown ## 📁 clip フォルダ
# @markdown CLIPモデル
clip_urls = "" # @param {type:"string"}

# @markdown ---
# @markdown ## 📁 upscale_models フォルダ
# @markdown アップスケールモデル
upscale_models_urls = "" # @param {type:"string"}

# @markdown ---
# @markdown ## 📁 embeddings フォルダ
# @markdown Embeddingsモデル
embeddings_urls = "" # @param {type:"string"}

# @markdown ---
# @markdown ## 📁 カスタムフォルダ（自由指定）
# @markdown フォルダパスとURLを指定
custom_folder_path = "" # @param {type:"string"}
custom_folder_urls = "" # @param {type:"string"}


# ========================================
# ダウンロード実行
# ========================================

print("=" * 70)
print("🚀 Starting Smart Download (Flux.2 & Wan 2.2 Ready)...")
print("=" * 70)

# 各フォルダへのダウンロード
process_urls(diffusion_models_urls, "/content/ComfyUI/models/diffusion_models")
process_urls(text_encoders_urls, "/content/ComfyUI/models/text_encoders")
process_urls(vae_urls, "/content/ComfyUI/models/vae")
process_urls(clip_vision_urls, "/content/ComfyUI/models/clip_vision")
process_urls(loras_urls, "/content/ComfyUI/models/loras")
process_urls(audio_encoders_urls, "/content/ComfyUI/models/audio_encoders")
process_urls(sam2_urls, "/content/ComfyUI/models/sam2")
process_urls(unet_urls, "/content/ComfyUI/models/unet")
process_urls(checkpoints_urls, "/content/ComfyUI/models/checkpoints")
process_urls(controlnet_urls, "/content/ComfyUI/models/controlnet")
process_urls(clip_urls, "/content/ComfyUI/models/clip")
process_urls(upscale_models_urls, "/content/ComfyUI/models/upscale_models")
process_urls(embeddings_urls, "/content/ComfyUI/models/embeddings")

# カスタムフォルダ
if custom_folder_path and custom_folder_urls.strip():
    process_urls(custom_folder_urls, custom_folder_path)

print("=" * 70)
print("✓ All downloads completed!")
print("=" * 70)


# ========================================
# ComfyUI起動設定
# ========================================

use_cloudflare = False # @param {type:"boolean"}
use_interface_in_cell = False # @param {type:"boolean"}

# 🆕 出力ディレクトリの設定
output_dir_arg = ""
if use_google_drive and enable_gdrive_output:
    output_dir_arg = f"--output-directory {GDRIVE_OUTPUT}"
    print("=" * 70)
    print(f"✅ ComfyUI output will be saved directly to Google Drive")
    print(f"📁 Output path: {GDRIVE_OUTPUT}")
    print("=" * 70)

if use_cloudflare:
    !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
    !dpkg -i cloudflared-linux-amd64.deb

    import subprocess
    import threading
    import time
    import socket
    import urllib.request

    def iframe_thread(port):
      while True:
          time.sleep(0.5)
          sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
          result = sock.connect_ex(('127.0.0.1', port))
          if result == 0:
            break
          sock.close()
      print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

      p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
      for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com " in l:
          print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    clear_output()

    threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

    if using_L4_GPU:
        command = f"python main.py {output_dir_arg} --cache-none --dont-print-server".strip()
        get_ipython().system(command)
    else:
        command = f"python main.py {output_dir_arg} --dont-print-server".strip()
        get_ipython().system(command)

elif use_interface_in_cell:
    import threading
    import time
    import socket
    def iframe_thread(port):
      while True:
          time.sleep(0.5)
          sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
          result = sock.connect_ex(('127.0.0.1', port))
          if result == 0:
            break
          sock.close()
      from google.colab import output
      output.serve_kernel_port_as_iframe(port, height=1024)
      clear_output()
      print("to open it in a window you can open this link here:")
      output.serve_kernel_port_as_window(port)

    threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

    if using_L4_GPU:
        command = f"python main.py {output_dir_arg} --cache-none --dont-print-server".strip()
        get_ipython().system(command)
    else:
        command = f"python main.py {output_dir_arg} --dont-print-server".strip()
        get_ipython().system(command)

else:
    import socket, time, threading
    from google.colab import output

    def link_thread(port):
        while True:
            time.sleep(0.5)
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            result = sock.connect_ex(('127.0.0.1', port))
            if result == 0:
                break
            sock.close()
        clear_output()
        print("Click the link below to launch the comfyui interface")
        output.serve_kernel_port_as_window(port)

    # Start thread for port 8188
    threading.Thread(target=link_thread, daemon=True, args=(8188,)).start()

    if using_L4_GPU:
        command = f"python main.py {output_dir_arg} --cache-none --dont-print-server".strip()
        get_ipython().system(command)
    else:
        command = f"python main.py {output_dir_arg} --dont-print-server".strip()
        get_ipython().system(command)

# AICU による解説

- [プロ志向画像生成 FLUX.2 最速ガイド! Google Colabノートブック付き](https://j.aicu.ai/251126)


# FLUX2関連

- [ComfyUI公式サポート](https://comfyanonymous.github.io/ComfyUI_examples/flux2/)

# 🌱📋各種モデル_DL元リポジトリのメモ🌱
###　下記リンクのリポジトリから必要なモデルファイルのダウンロードURLを取得してきてください。
---
#🎬Wan2.2用 Diffusion Model
##＜ti2v‗5B‗fp16＞ ※標準テンプレートに対応
####※下記リンクをそのまま使用可能
- wan2.2_ti2v_5B_fp16：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_ti2v_5B_fp16.safetensors)

##＜t2v‗14B‗fp8＞ ※標準テンプレートに対応
####※下記リンクをそのまま使用可能
- Wan2.2-T2V-A14B(High‗noise)：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_t2v_high_noise_14B_fp8_scaled.safetensors)
- Wan2.2-T2V-A14B(Low‗noise)：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_t2v_low_noise_14B_fp8_scaled.safetensors)
##＜i2v‗14B‗fp8＞ ※標準テンプレートに対応
####※下記リンクをそのまま使用可能
- Wan2.2-I2V-A14B(High‗noise)：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_i2v_high_noise_14B_fp8_scaled.safetensors)
- Wan2.2-I2V-A14B(Low‗noise)：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_i2v_low_noise_14B_fp8_scaled.safetensors)

##＜t2v‗GGUF＞ ※ModelLoaderの変更が必要
####※下記リンクをそのまま使用可能（GGUF_Q8）
- Wan2.2-T2V-A14B-HighNoise-Q8_0.gguf：(https://huggingface.co/QuantStack/Wan2.2-T2V-A14B-GGUF/resolve/main/HighNoise/Wan2.2-T2V-A14B-HighNoise-Q8_0.gguf)
- Wan2.2-T2V-A14B-LowNoise-Q8_0.gguf：(https://huggingface.co/QuantStack/Wan2.2-T2V-A14B-GGUF/resolve/main/LowNoise/Wan2.2-T2V-A14B-LowNoise-Q8_0.gguf)

####※下記リポジトリ内のモデルファイルのダウンロードURLを任意に取得
- Wan2.2-T2V-A14B-GGUF：https://huggingface.co/QuantStack/Wan2.2-T2V-A14B-GGUF/tree/main
##＜i2v‗GGUF＞ ※ModelLoaderの変更が必要
####※下記リンクをそのまま使用可能（GGUF_Q8）
- Wan2.2-I2V-A14B-HighNoise-Q8_0.gguf：(https://huggingface.co/QuantStack/Wan2.2-I2V-A14B-GGUF/resolve/main/HighNoise/Wan2.2-I2V-A14B-HighNoise-Q8_0.gguf)
- Wan2.2-I2V-A14B-LowNoise-Q8_0.gguf：(https://huggingface.co/QuantStack/Wan2.2-I2V-A14B-GGUF/resolve/main/LowNoise/Wan2.2-I2V-A14B-LowNoise-Q8_0.gguf)

####※下記リポジトリ内のモデルファイルのダウンロードURLを任意に取得
- Wan2.2-I2V-A14B-GGUF：(https://huggingface.co/QuantStack/Wan2.2-I2V-A14B-GGUF/tree/main)

##＜Fun&Inp＞ ※動画の人物の動きを再現
####※下記リポジトリ内のモデルファイルのダウンロードURLを任意に取得
- Wan2.2-Fun＆Inpl：(https://huggingface.co/Kijai/WanVideo_comfy_fp8_scaled/tree/main/Fun)

##＜Animate‗GGUF＞ ※動画中の被写体をすげ替えor参照画像に反映
####※下記リポジトリ内のモデルファイルのダウンロードURLを任意に取得
- Wan2.2-Animate-A14B-GGUF(https://huggingface.co/QuantStack/Wan2.2-Animate-14B-GGUF/tree/main)

#🎬Wan2.2用LoRA
##＜lightx2v_4steps（高速化）＞※High＆Low 別々に指定
####※下記リンクをそのまま使用可能
- LightX2v_t2v-4step_High：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/wan2.2_t2v_lightx2v_4steps_lora_v1.1_high_noise.safetensors)
- LightX2v_t2v-4step_Low：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/wan2.2_t2v_lightx2v_4steps_lora_v1.1_low_noise.safetensors)
- LightX2v_i2v-4step_High：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/wan2.2_i2v_lightx2v_4steps_lora_v1_high_noise.safetensors)
- LightX2v_i2v-4step_Low：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/wan2.2_i2v_lightx2v_4steps_lora_v1_low_noise.safetensors)
##＜lightx2v_Rank（高速化）＞※High＆Low共通
####※下記リンクをそのまま使用可能
- lightx2v_I2V_14B_480p_cfg_step_distill_rank128_bf16：(https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Lightx2v/lightx2v_I2V_14B_480p_cfg_step_distill_rank128_bf16.safetensors)
- lightx2v_T2V_14B_cfg_step_distill_v2_lora_rank128_bf16：(https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Lightx2v/lightx2v_T2V_14B_cfg_step_distill_v2_lora_rank128_bf16.safetensors)

- LightX2v_t2v&i2v-Rank：(https://huggingface.co/Kijai/WanVideo_comfy/tree/main/Lightx2v)

##＜relight_LoRA（Animate用）＞ ※Wan2.2 Animateに必須のLoRA
####※下記リンクをそのまま使用可能
- Wan2.2_relight：(https://huggingface.co/Kijai/WanVideo_comfy/tree/main/LoRAs/Wan22_relight)

#🎬Wan2.2用VAE
##＜VAE＞
####※下記リンクをそのまま使用可能
- Wan2.1_VAE.safetensors：(https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/vae/wan_2.1_vae.safetensors)
- wan2.2_vae.safetensors：(https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/vae/wan2.2_vae.safetensors)

#🎬Wan2.2用TextEncoder
##＜TextEncoder＞
####※下記リポジトリ内のモデルファイルのダウンロードURLを任意に取得
- umt5_xxl_fp16.safetensors：
(https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/tree/main/split_files/text_encoders)

#🎬Wan2.2用ClipVision
##＜ClipVision＞
####※下記リポジトリ内のモデルファイルのダウンロードURLを任意に取得
- clip_vision_h.safetensors：(https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/tree/main/split_files/clip_vision)

#🖼 StableDiffsions XL
##＜Checkpoint＞
Sierunami.v1：(https://civitai.com/api/download/models/1176276)
※CIVITAIのAPIキーが必要

mellow_pencil-XL-v1.0.0-base：(https://huggingface.co/bluepen5805/mellow_pencil-XL/resolve/main/mellow_pencil-XL-v1.0.0-base.safetensors)

mellow_pencil-XL-v1.0.0-chaos：(https://huggingface.co/bluepen5805/mellow_pencil-XL/resolve/main/mellow_pencil-XL-v1.0.0-chaos.safetensors)

##＜VAE＞
####※下記リンクをそのまま使用可能
sdxl_vae：(https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors)

---


# 謝辞：このノートブックは 道草雑草子さんから許諾を受けて AICU Japanがメンテナンスしております。質問は [X@AICUai](https://x.com/AICUai) まで。

# 🍀 原作者のざすこさんの「はじめに」より 🍀
# 😊いつもチャンネルをご覧いただいている皆様、ご視聴ありがとうございます！  
### 🌱このノートブックは主に、生成AIツール活用の研究目的のために制作しました。
### 🌱使用目的は「ComfyUI」上で「Wan2.2」等のツールを高価なPC不要でも使用可能にする為に制作したものです。
### 🌱各種生成ツールのコツなど詳しい使い方については、私のYoutbeチャンネルの中で解説しています。
### 🌱より便利で汎用的なノートブックやＷＦも「メンバーシップ特典」として配布しています。
### 📺『ＡＩみちくさｃｈ』 (https://www.youtube.com/channel/UC84fyKjiilxssZVxhE_RiaA)
####　🍀制作者：ざすこ（道草 雑草子） Xアカウント：https://x.com/zasuko_michiksa
##### 🥰このノートブックが役に立ったよ～！と思った方は、チャンネル登録や、メンバーシップのご参加をご検討頂けますと幸いです。
#＜このnotebookの使い方の解説動画＞
### https://www.youtube.com/watch?v=LdxfzWRe2sY